# Análise Comparativa de Modelos de Machine Learning
## Classificação de Depressão em Estudantes

### Objetivo do Projeto
Este projeto implementa e compara três diferentes algoritmos de machine learning para classificação binária de depressão em estudantes:
- **Support Vector Machine (SVM)**
- **Árvore de Decisão**
- **Redes Neurais**

### Metodologia
- Dataset balanceado (50% / 50%)
- Divisão treino/teste: 80% / 20%
- Validação cruzada para otimização de hiperparâmetros
- Métricas: Accuracy, Precision, Recall, F1-Score

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Resultados dos Modelos

### Consolidação dos Resultados Obtidos
Os resultados abaixo foram coletados dos notebooks individuais de cada modelo:

In [3]:
svm_results = {
    'Modelo': 'SVM',
    'Accuracy': 0.83,
    'Precision_0': 0.84, 
    'Recall_0': 0.81,
    'F1_Score_0': 0.82,
    'Precision_1': 0.81,
    'Recall_1': 0.84,
    'F1_Score_1': 0.83,
    'Ein': 0.1689,  
    'Eout': 0.1749 
}

arvore_results = {
    'Modelo': 'Árvore de Decisão',
    'Accuracy': 0.82,
    'Precision_0': 0.81,
    'Recall_0': 0.82,
    'F1_Score_0': 0.82,
    'Precision_1': 0.83,
    'Recall_1': 0.81,
    'F1_Score_1': 0.82,
    'Ein': 0.1790,  
    'Eout': 0.1824 
}

redes_results = {
    'Modelo': 'Redes Neurais',
    'Accuracy': 0.83, 
    'Precision_0': 0.82,
    'Recall_0': 0.84,
    'F1_Score_0': 0.83,
    'Precision_1': 0.83,
    'Recall_1': 0.82,
    'F1_Score_1': 0.82,
    'Ein': 0.1787,
    'Eout': 0.1669
}

results_df = pd.DataFrame([svm_results, arvore_results, redes_results])
print("=== RESULTADOS CONSOLIDADOS ===")
print(results_df[['Modelo', 'Accuracy', 'F1_Score_0', 'F1_Score_1','Recall_0', 'Recall_1']])

=== RESULTADOS CONSOLIDADOS ===
              Modelo  Accuracy  F1_Score_0  F1_Score_1  Recall_0  Recall_1
0                SVM      0.83        0.82        0.83      0.81      0.84
1  Árvore de Decisão      0.82        0.82        0.82      0.82      0.81
2      Redes Neurais      0.83        0.83        0.82      0.84      0.82


In [5]:
# Análise detalhada dos modelos
print("=== ANÁLISE COMPARATIVA DETALHADA ===\n")

# 1. Comparação das métricas principais
print("1. MÉTRICAS PRINCIPAIS:")
print("-" * 50)
for index, row in results_df.iterrows():
    print(f"{row['Modelo']:<20} | Accuracy: {row['Accuracy']:.3f} | F1-Score Médio: {(row['F1_Score_0'] + row['F1_Score_1'])/2:.3f}")

# 2. Análise de overfitting (Ein vs Eout)
print("\n2. ANÁLISE DE OVERFITTING (Ein vs Eout):")
print("-" * 50)
results_df['Overfitting'] = results_df['Eout'] - results_df['Ein']
for index, row in results_df.iterrows():
    status = "✓ Bom" if abs(row['Overfitting']) < 0.02 else "⚠ Overfitting" if row['Overfitting'] > 0.02 else "⚠ Underfitting"
    print(f"{row['Modelo']:<20} | Ein: {row['Ein']:.4f} | Eout: {row['Eout']:.4f} | Diff: {row['Overfitting']:.4f} | {status}")

# 3. Balanceamento entre classes
print("\n3. BALANCEAMENTO ENTRE CLASSES:")
print("-" * 50)
for index, row in results_df.iterrows():
    f1_diff = abs(row['F1_Score_0'] - row['F1_Score_1'])
    balance_status = "✓ Balanceado" if f1_diff < 0.02 else "⚠ Desbalanceado"
    print(f"{row['Modelo']:<20} | F1 Classe 0: {row['F1_Score_0']:.3f} | F1 Classe 1: {row['F1_Score_1']:.3f} | Diff: {f1_diff:.3f} | {balance_status}")

# 4. Cálculo do score final ponderado
print("\n4. SCORE FINAL PONDERADO:")
print("-" * 50)

def calculate_final_score(row):
    # Peso para accuracy (30%)
    accuracy_score = row['Accuracy'] * 0.3
    
    # Peso para F1-Score médio (40%)
    f1_mean = (row['F1_Score_0'] + row['F1_Score_1']) / 2
    f1_score = f1_mean * 0.4
    
    # Penalidade por overfitting (20%)
    overfitting_penalty = max(0, 0.2 - abs(row['Overfitting']) * 10) * 0.2
    
    # Penalidade por desbalanceamento (10%)
    balance_penalty = max(0, 0.1 - abs(row['F1_Score_0'] - row['F1_Score_1']) * 5) * 0.1
    
    return accuracy_score + f1_score + overfitting_penalty + balance_penalty

results_df['Score_Final'] = results_df.apply(calculate_final_score, axis=1)

# Ordenar por score final
results_df_sorted = results_df.sort_values('Score_Final', ascending=False)

for index, row in results_df_sorted.iterrows():
    print(f"{row['Modelo']:<20} | Score Final: {row['Score_Final']:.4f}")

# 5. Determinação do melhor modelo
print(f"\n{'='*60}")
print("RESULTADO FINAL:")
print(f"{'='*60}")

best_model = results_df_sorted.iloc[0]
print(f"MELHOR MODELO: {best_model['Modelo']}")
print(f"Score Final: {best_model['Score_Final']:.4f}")
print(f"Accuracy: {best_model['Accuracy']:.3f}")
print(f"F1-Score Médio: {(best_model['F1_Score_0'] + best_model['F1_Score_1'])/2:.3f}")
print(f"Overfitting: {best_model['Overfitting']:.4f} {'(Controlado)' if abs(best_model['Overfitting']) < 0.02 else '(Presente)'}")

# 6. Justificativa da escolha
print("\nJUSTIFICATIVA:")
print("-" * 60)

if best_model['Modelo'] == 'SVM':
    print("✓ Excelente accuracy e F1-score")
    print("✓ Boa generalização (Ein vs Eout)")
    print("✓ Balanceamento adequado entre classes")
elif best_model['Modelo'] == 'Redes Neurais':
    print("✓ Melhor generalização (menor Eout)")
    print("✓ Boa performance geral")
    print("✓ Flexibilidade para dados complexos")
else:
    print("✓ Interpretabilidade do modelo")
    print("✓ Simplicidade e eficiência")
    print("✓ Boa performance geral")

print(f"\nCONCLUSÃO: O modelo {best_model['Modelo']} é o mais adequado para este problema de classificação de depressão.")

=== ANÁLISE COMPARATIVA DETALHADA ===

1. MÉTRICAS PRINCIPAIS:
--------------------------------------------------
SVM                  | Accuracy: 0.830 | F1-Score Médio: 0.825
Árvore de Decisão    | Accuracy: 0.820 | F1-Score Médio: 0.820
Redes Neurais        | Accuracy: 0.830 | F1-Score Médio: 0.825

2. ANÁLISE DE OVERFITTING (Ein vs Eout):
--------------------------------------------------
SVM                  | Ein: 0.1689 | Eout: 0.1749 | Diff: 0.0060 | ✓ Bom
Árvore de Decisão    | Ein: 0.1790 | Eout: 0.1824 | Diff: 0.0034 | ✓ Bom
Redes Neurais        | Ein: 0.1787 | Eout: 0.1669 | Diff: -0.0118 | ✓ Bom

3. BALANCEAMENTO ENTRE CLASSES:
--------------------------------------------------
SVM                  | F1 Classe 0: 0.820 | F1 Classe 1: 0.830 | Diff: 0.010 | ✓ Balanceado
Árvore de Decisão    | F1 Classe 0: 0.820 | F1 Classe 1: 0.820 | Diff: 0.000 | ✓ Balanceado
Redes Neurais        | F1 Classe 0: 0.830 | F1 Classe 1: 0.820 | Diff: 0.010 | ✓ Balanceado

4. SCORE FINAL PONDERAD